In [1]:
import pandas as pd
import json
import os

In [2]:
import tensorflow as tf

# Get the list of available GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        # Set memory growth to False for each GPU
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, False)
        
        # Log how much memory is allocated
        print("All available GPU memory will be allocated.")
    except RuntimeError as e:
        # Memory growth must be set before initializing GPUs
        print(e)

2024-11-15 17:09:16.276602: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731672556.290983   38378 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731672556.295106   38378 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 17:09:16.308851: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


All available GPU memory will be allocated.


In [7]:
json_files = ["/home/sohail/projects/NLP_project/train.json", "/home/sohail/projects/NLP_project/test_freq.json", "/home/sohail/projects/NLP_project/test_rare.json", "/home/sohail/projects/NLP_project/valid_freq.json", "/home/sohail/projects/NLP_project/valid_rare.json"]
data=[]

In [8]:
def process_json(file):
    with open(file, 'r') as f:
        conversations = json.load(f)  # Load the JSON data

        # Iterate over each conversation in the file
        for conv_id, conv_data in conversations.items():
            content = conv_data['content']  # Access the conversation turns
            for turn in content:
                # Extract relevant fields from each conversation turn
                message = turn.get('message', '')
                agent = turn.get('agent', '')
                sentiment = turn.get('sentiment', '')
                turn_rating = turn.get('turn_rating', '')
                knowledge_source = turn.get('knowledge_source', [])

                # Append the structured data as a row
                data.append({
                    'message': message,
                    'agent': agent,
                    'sentiment': sentiment,
                    'turn_rating': turn_rating,
                    'knowledge_source': ', '.join(knowledge_source)  # Join list as a string
                })


In [9]:
for json_file in json_files:
    process_json(json_file)

In [10]:
df = pd.DataFrame(data)
df.to_csv('structured_conversations.csv', index=False)
print(df.head)

<bound method NDFrame.head of                                                   message    agent  \
0                   Are you a fan of Google or Microsoft?  agent_1   
1       Both are excellent technology they are helpful...  agent_2   
2       I'm not  a huge fan of Google, but I use it a ...  agent_1   
3       Google provides online related services and pr...  agent_2   
4       Yeah, their services are good. I'm just not a ...  agent_1   
...                                                   ...      ...   
235276  I didn't know that and I wouldn't like to be t...  agent_1   
235277  I am both terrified and amazed of them , maybe...  agent_2   
235278  lol  Sharks evolved earlier than dinosaurs and...  agent_1   
235279                well I love to talk about animals!!  agent_2   
235280                         Me too.  Have a great day!  agent_1   

                     sentiment turn_rating knowledge_source  
0       Curious to dive deeper        Good              FS1  
1    

In [3]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, processors
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import NFKC
from tokenizers.processors import BertProcessing
from tokenizers.trainers import BpeTrainer
import pandas as pd

In [9]:
df = pd.read_csv('structured_conversations.csv')
texts = df['message'].tolist()
tokenizer = Tokenizer(models.BPE())
tokenizer.normalizer = NFKC()
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(vocab_size=10000, special_tokens=["<s>", "<pad>", "</s>", "<unk>"])
tokenizer.train_from_iterator(texts, trainer=trainer)
tokenizer.post_processor = BertProcessing(("</s>", tokenizer.token_to_id("</s>")),
                                          ("<s>", tokenizer.token_to_id("<s>")))
tokenizer.save("bpe_tokenizer.json")
encoded = tokenizer.encode("Both are excellent technology they are helpful in many ways.")
print("Tokenized IDs:", encoded.ids)
print("Tokenized Tokens:", encoded.tokens)
df['tokenized_message'] = df['message'].apply(lambda x: tokenizer.encode(x).tokens)
df.to_csv('tokenized_conversations.csv', index=False)




Tokenized IDs: [0, 5467, 183, 3847, 1663, 189, 183, 5225, 126, 444, 540, 18, 2]
Tokenized Tokens: ['<s>', 'Both', 'are', 'excellent', 'technology', 'they', 'are', 'helpful', 'in', 'many', 'ways', '.', '</s>']


In [10]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers
import pandas as pd


In [11]:
df = pd.read_csv('structured_conversations.csv')
texts = df['message'].tolist()
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
trainer = trainers.BpeTrainer(vocab_size=20000, special_tokens=["<s>", "<pad>", "</s>", "<unk>"])
tokenizer.train_from_iterator(texts, trainer=trainer)
tokenizer.save("bpe_tokenizer.json")


In [12]:
import tensorflow as tf
from tokenizers import Tokenizer
import numpy as np
import pandas as pd

In [30]:
tokenizer = Tokenizer.from_file("bpe_tokenizer.json")
df = pd.read_csv('structured_conversations.csv')
def tokenize_texts(texts, tokenizer, max_length=130):
    tokenized = []
    for text in texts:
        tokens = tokenizer.encode(text).ids
        # Truncate (post-truncation) if longer than max_length
        if len(tokens) > max_length:
            tokens = tokens[:max_length]
        tokenized.append(tokens)
    return tokenized
texts = df['message'].tolist()
tokenized_sequences = tokenize_texts(texts, tokenizer, max_length=130)
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(
    tokenized_sequences, maxlen=130, padding='pre', truncating='post', value=tokenizer.token_to_id("<pad>")
)
attention_masks = np.where(padded_sequences != tokenizer.token_to_id("<pad>"), 1, 0)
input_sequences = tf.convert_to_tensor(padded_sequences)
attention_masks = tf.convert_to_tensor(attention_masks)
batch_size = 32  # Set your batch size
dataset = tf.data.Dataset.from_tensor_slices((input_sequences, attention_masks))
dataset = dataset.shuffle(buffer_size=len(input_sequences)).batch(batch_size)
for batch_input, batch_mask in dataset.take(1):
    print("Input batch shape:", batch_input.shape)
    print("Attention mask batch shape:", batch_mask.shape)

Input batch shape: (32, 130)
Attention mask batch shape: (32, 130)


In [14]:
import numpy as np
import tensorflow as tf

In [15]:
glove_file_path = '/home/sohail/projects/NLP_project/glove.6B.200d.txt'
embedding_dim = 200
vocab_size = 20000  # Ensure this matches the size of your tokenizer's vocabulary
glove_embeddings_index = {}
with open(glove_file_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        glove_embeddings_index[word] = vector

print(f"Loaded {len(glove_embeddings_index)} word vectors from GloVe.")

Loaded 400000 word vectors from GloVe.


In [18]:
embedding_matrix = np.zeros((vocab_size, embedding_dim))
tokenizer = tf.keras.preprocessing.text.Tokenizer()
for word, index in tokenizer.word_index.items():
    if index < vocab_size:
        embedding_vector = glove_embeddings_index.get(word)
        if embedding_vector is not None:
            # Words found in GloVe will have their embeddings loaded
            embedding_matrix[index] = embedding_vector
        else:
            # Words not found in GloVe will be left as zeros or can be randomly initialized
            embedding_matrix[index] = np.random.normal(size=(embedding_dim,))

print(f"Created an embedding matrix of shape {embedding_matrix.shape}.")

Created an embedding matrix of shape (20000, 200).


In [19]:
print(embedding_matrix)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [23]:
embedding_layer = tf.keras.layers.Embedding(
    input_dim=vocab_size,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    input_length=130,  # Maximum sequence length
    trainable=False  # Set to False if you don't want to update the GloVe embeddings during training
)


In [ ]:
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from datasets import Dataset
import pandas as pd

# Load GPT-2 tokenizer from Hugging Face
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set PAD token to EOS token

# Load GPT-2 model
model = TFGPT2LMHeadModel.from_pretrained('gpt2')

# Maximum sequence length and batch size
MAX_SEQ_LENGTH = 130
BATCH_SIZE = 8

# Function to tokenize and encode input text
def tokenize_function(examples):
    return tokenizer(
        examples["message"],
        padding="max_length",
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
        return_tensors='tf',  # Ensure to return tensors for TensorFlow
    )

# Load your dataset (assuming it's saved as a CSV file)
df = pd.read_csv('/home/sohail/projects/NLP_project/structured_conversations.csv')

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['agent', 'sentiment', 'turn_rating', 'knowledge_source'])

# Convert tokenized dataset to TensorFlow format
def generate_tf_dataset(tokenized_data, batch_size, shuffle=True):
    input_ids = tokenized_data['input_ids']
    attention_masks = tokenized_data['attention_mask']
    
    dataset = tf.data.Dataset.from_tensor_slices(({
        'input_ids': input_ids, 'attention_mask': attention_masks
    }, input_ids))  # Model needs inputs and labels (here labels are same as input_ids)

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(input_ids))
    
    dataset = dataset.batch(batch_size)
    return dataset

# Create TensorFlow dataset for training
tf_dataset = generate_tf_dataset(tokenized_dataset, BATCH_SIZE)


In [15]:
import tensorflow as tf

# Define loss manually if compute_loss is causing issues
def custom_loss(y_true, y_pred):
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    return loss_fn(y_true, y_pred)

# Compile the model with a custom loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer, loss=custom_loss)

# Ensure the dataset includes both inputs and labels
history = model.fit(
    tf_dataset,
    epochs=20,
    verbose=1
)


Epoch 1/20


2024-11-15 17:27:02.600684: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:497] Allocator (GPU_0_bfc) ran out of memory trying to allocate 12.19MiB (rounded to 12779520)requested by op tfgpt2lm_head_model_8/transformer/h_._0/mlp/dropout_299/dropout/random_uniform/RandomUniform
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-11-15 17:27:02.600760: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1053] BFCAllocator dump for GPU_0_bfc
2024-11-15 17:27:02.600775: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1060] Bin (256): 	Total Chunks: 64, Chunks in use: 64. 16.0KiB allocated for chunks. 16.0KiB in use in bin. 285B client-requested in use in bin.
2024-11-15 17:27:02.600780: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1060] Bin (512): 	Total Chunks: 1, Chunks in use: 1. 768B allocated for 

ResourceExhaustedError: Graph execution error:

Detected at node tfgpt2lm_head_model_8/transformer/h_._0/mlp/dropout_299/dropout/random_uniform/RandomUniform defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/sohail/projects/venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/sohail/projects/venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/sohail/projects/venv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/sohail/projects/venv/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/usr/lib/python3.12/asyncio/base_events.py", line 1987, in _run_once

  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run

  File "/home/